# Meta Motivo Tutorial
This notebook provides a simple introduction on how to use the Meta Motivo api.

## All imports

In [1]:
from metamotivo.fb_cpr.huggingface import FBcprModel
from huggingface_hub import hf_hub_download
from humenv import make_humenv
from gymnasium.wrappers import FlattenObservation, TransformObservation
from metamotivo.buffers.buffers import DictBuffer
from humenv.env import make_from_name
from humenv import rewards as humenv_rewards

import torch
import mediapy as media
import math
import h5py
from pathlib import Path
import numpy as np

/root/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model download
The first step is to download the model. We show how to use HuggingFace hub for that.

In [2]:
# FB-CPR (Forward-Backward representations with Conditional Policy Regularization)
model = FBcprModel.from_pretrained("/workspace/metamotivo/weights/metamotivo-M-1")
# print(model)
device = "cpu"
env, _ = make_humenv(
    num_envs=1,
    wrappers=[
        FlattenObservation,
        lambda env: TransformObservation(
            env, lambda obs: torch.tensor(obs.reshape(1, -1), dtype=torch.float32, device=device), None
        ),
    ],
    state_init="Default",
)

model.to(device)
z = model.sample_z(1)
print(f"embedding size {z.shape}")
print(f"z norm: {torch.norm(z)}")
print(f"z norm / sqrt(d): {torch.norm(z) / math.sqrt(z.shape[-1])}")
observation, _ = env.reset()

Loading weights from local directory
embedding size torch.Size([1, 256])
z norm: 15.999999046325684
z norm / sqrt(d): 0.9999999403953552


**Run a policy from Meta Motivo:**

Now that we saw how to load a pre-trained Meta Motivo policy, we can prompt it and execute actions with it. 

The first step is to sample a context embedding `z` that needs to be passed to the policy.

In [8]:
# frames = [env.render()]
for i in range(10):
    action = model.act(observation, z, mean=True)
    observation, reward, terminated, truncated, info = env.step(action.cpu().numpy().ravel())
    # frames.append(env.render())
    print()

# media.show_video(frames, fps=30)


In [18]:
print(type(action))
print(action)

<class 'torch.Tensor'>
tensor([[ 0.9035,  0.0891, -0.5720, -0.6155,  0.9966,  0.9993,  0.1562,  1.0000,
          0.9818,  0.5814,  0.9950,  1.0000,  0.2559,  0.0516,  0.3607, -0.9981,
          1.0000, -0.9619, -0.6218, -0.7449,  0.9997,  0.7061, -1.0000,  1.0000,
         -0.9929,  0.0726,  0.1072, -0.2759, -0.1077,  0.4683, -0.6187, -0.0305,
          0.2818, -0.2209, -0.1873,  0.6845, -1.0000,  0.9999,  0.9444, -1.0000,
         -0.4481, -0.6869, -0.1910, -0.6637, -0.6141, -1.0000,  0.9961,  1.0000,
         -0.9733,  0.9992, -1.0000,  1.0000, -0.2173, -0.3821, -1.0000,  1.0000,
          0.9992, -0.1799,  0.6509,  0.9959,  0.9732, -0.9979, -1.0000,  1.0000,
          0.2384, -1.0000, -1.0000, -0.0085, -0.3907]])


In [12]:
print(type(observation))
print(observation)

<class 'torch.Tensor'>
tensor([[ 9.0288e-01, -3.5818e-02,  7.4090e-02, -8.0363e-02,  8.9125e-02,
          1.2323e-01, -4.3242e-01,  1.1802e-01,  1.1960e-01, -8.3199e-01,
          2.1010e-01,  2.0011e-01, -8.8750e-01, -3.5517e-02, -6.2976e-02,
         -8.6975e-02, -9.8935e-02, -1.4523e-01, -4.5730e-01, -2.2997e-01,
         -2.0895e-01, -8.3084e-01, -1.0646e-01, -2.4749e-01, -8.6852e-01,
          1.1301e-02, -8.0794e-03,  1.1139e-01,  2.3131e-02,  1.1292e-02,
          2.4479e-01,  6.0650e-02,  2.1195e-02,  2.8883e-01,  6.6724e-02,
          4.7383e-02,  5.0535e-01,  1.3728e-01,  7.1345e-02,  5.4184e-01,
          4.0714e-02,  1.1323e-01,  4.0423e-01,  8.6296e-03,  2.0259e-01,
          3.8764e-01, -3.9156e-02,  2.0912e-01,  1.3077e-01, -1.2484e-01,
          2.0344e-01, -1.0335e-01, -1.5902e-01,  1.6669e-01, -1.7282e-01,
          6.3904e-02, -4.4696e-02,  4.2278e-01,  1.0551e-01, -1.3759e-01,
          4.2391e-01,  1.3916e-01, -1.0366e-01,  1.7348e-01,  1.8823e-01,
         -1.382

In [14]:
print(type(reward))
print(reward)

<class 'float'>
0.0


In [15]:
print(type(terminated))
print(terminated)

<class 'bool'>
False


In [16]:
print(type(truncated))
print(truncated)

<class 'bool'>
False


In [17]:
print(type(info))
print(info)

<class 'dict'>
{'qpos': array([ 2.04939863e-01, -1.48138044e-01,  9.02875855e-01,  5.69595128e-01,
        8.12703314e-01,  8.46006055e-02,  8.89800556e-02, -6.98218788e-01,
        7.49476493e-02,  9.66407166e-04,  1.80792690e-01,  1.15077368e-01,
        1.05973054e-02,  1.80115151e-01,  3.34181288e-01, -1.63914815e-02,
       -2.11745392e-02,  8.68783532e-02,  7.75431692e-03, -2.04554534e-01,
       -2.23168972e-02, -1.66907154e-01,  8.91311615e-02,  8.48458267e-02,
       -8.24864300e-02, -2.75055398e-01, -1.65494532e-01,  7.95946600e-02,
        1.09813856e-01, -7.68003545e-02,  7.85224809e-02, -2.41032977e-01,
        1.16813197e-01, -1.52958112e-01,  1.70477761e-01, -1.30603084e-04,
        1.84186332e-04, -2.48355583e-02,  5.35488345e-02, -4.84043163e-02,
        1.89608907e-01,  1.22325826e-01,  7.71720487e-02, -1.14433942e-01,
        2.40042734e-01,  2.50472267e-01, -1.20461346e-01,  6.48323439e-02,
       -3.92148523e-01,  2.62719547e-01, -5.00806403e-01, -9.19044666e-01,
 

In [ ]:
print()

### Computing Q-functions

FB-CPR provides a way of directly computing the action-value function of any policy embedding `z` on any task embedding `z_r`. Then, the Q function of a policy $z$ is given by

$Q(s,a, z) = F(s,a,z) \cdot z_r$

The task embedding can be computed in the following way. Given a set of samples labeled with rewards $(s,a,s',r)$, the task embedding is given by: 

$z_r = \mathrm{normalised}(\sum_{i \in \mathrm{batch}} r_i B(s'_i))$.

In [19]:
def Qfunction(state, action, z_reward, z_policy):
    F = model.forward_map(obs=state, z=z_policy.repeat(state.shape[0],1), action=action) # num_parallel x num_samples x z_dim
    Q = F @ z_reward.ravel()
    return Q.mean(axis=0)

z_reward = model.sample_z(1)
z_policy = model.sample_z(1)
state = torch.rand((10, env.observation_space.shape[0]), device=model.cfg.device, dtype=torch.float32)
action = torch.rand((10, env.action_space.shape[0]), device=model.cfg.device, dtype=torch.float32)*2 - 1
Q = Qfunction(state, action, z_reward, z_policy)
print(Q)

tensor([255.3591, 200.0598, 198.1490, 198.0176, 198.7776, 212.7530, 197.3234,
        219.2996, 223.4091, 198.2895])


## Prompting the model

We have seen that we can condition the model via the context variable `z`. We can control the task to execute via _prompting_ (or _policy inference_).

### Reward prompts
The first version of inference we investigate is the reward prompting, i.e., given a set of reward label samples we can infer in a zero-shot way the near-optimal policy for solving such task.

First step, download the data for inference. We provide a buffer for inference of about 500k samples. This buffer has been generated by randomly subsampling the final replay buffer.

In [ ]:
local_dir = "metamotivo-S-1-datasets"
dataset = "buffer_inference_500000.hdf5"
buffer_path = hf_hub_download(
        repo_id="facebook/metamotivo-S-1",
        filename=f"data/{dataset}",
        repo_type="model",
        local_dir=local_dir,
    )
print(buffer_path)

hf = h5py.File(buffer_path, "r")
print(hf.keys())
data = {}
for k, v in hf.items():
    print(f"{k:20s}: {v.shape}")
    data[k] = v[:]
buffer = DictBuffer(capacity=data["qpos"].shape[0], device="cpu")
buffer.extend(data)
del data

Now that we have download the h5 file for inference, we can conveniently loaded it in a buffer.

In [ ]:
batch = buffer.sample(5)
for k, v in batch.items():
    print(f"{k:20s}: {v.shape}")

As you can see, the buffer does not provide a reward signal. We need to label this buffer with the desired reward function. We provide API for that but here we start looking into the basic steps:
* Instantiate a reward function
* Computing the reward from the batch data

In [ ]:
reward_fn = humenv_rewards.LocomotionReward(move_speed=2.0) # move ahead with speed 2
# humenv provides also a name-base reward initialization. We could
# get the same reward function in this way
reward_fn = make_from_name("move-ego-0-2") 
print(reward_fn)

We can call the method `__call__` to obtain a reward value from the physics state. This function receives a mujoco model, qpos, qvel and the action. See the humenv tutorial for more information.

In [ ]:
N = 100_000
batch = buffer.sample(N)
rewards = []
for i in range(N):
    rewards.append(
        reward_fn(
            env.unwrapped.model,
            qpos=batch["next_qpos"][i],
            qvel=batch["next_qvel"][i],
            ctrl=batch["action"][i])
    )
rewards = np.stack(rewards).reshape(-1,1)
print(rewards.ravel())

**Note** that the reward functions implemented in humenv are functions of next state and action which means we need to use `next_qpos` and `next_qvel` that are the physical state of the system at the next state.

We provide a multi-thread version for faster relabeling, see `metamotivo.wrappers.humenvbench.relabel`.

In [ ]:
from metamotivo.wrappers.humenvbench import relabel
rewards = relabel(
    env,
    qpos=batch["next_qpos"],
    qvel=batch["next_qvel"],
    action=batch["action"],
    reward_fn=reward_fn, 
    max_workers=8
)
print(rewards.ravel())

We can now infer the context `z` for the selected task.

In [ ]:
z = model.reward_wr_inference(
    next_obs=batch["next_observation"],
    reward=torch.tensor(rewards, device=model.cfg.device, dtype=torch.float32)
)
print(z.shape)

observation, _ = env.reset()
frames = [env.render()]
for i in range(10):
    action = model.act(observation, z, mean=True)
    observation, reward, terminated, truncated, info = env.step(action.cpu().numpy().ravel())
    frames.append(env.render())

media.show_video(frames, fps=30)

Let's compute the **Q-function** for this policy.

In [ ]:
z_reward = torch.sum(
    model.backward_map(obs=batch["next_observation"]) * torch.tensor(rewards, dtype=torch.float32, device=model.cfg.device),
    dim=0
)
z_reward = model.project_z(z_reward)
Q = Qfunction(batch["observation"], batch["action"], z_reward, z)
print(Q)

# Goal and Tracking prompts
The model supports two other modalities, `goal` and `tracking`. These two modalities expose similar functions for context inference:
- `def goal_inference(self, next_obs: torch.Tensor) -> torch.Tensor`
- `def tracking_inference(self, next_obs: torch.Tensor) -> torch.Tensor`
  
We show an example on how to perform goal inference.

In [ ]:
goal_qpos = np.array([0.13769039,-0.20029453,0.42305034,0.21707786,0.94573617,0.23868944
,0.03856998,-1.05566834,-0.12680767,0.11718296,1.89464102,-0.01371153
,-0.07981451,-0.70497424,-0.0478,-0.05700732,-0.05363342,-0.0657329
,0.08163511,-1.06263979,0.09788937,-0.22008936,1.85898192,0.08773695
,0.06200327,-0.3802791,0.07829525,0.06707749,0.14137152,0.08834448
,-0.07649805,0.78328658,0.12580912,-0.01076061,-0.35937259,-0.13176489
,0.07497022,-0.2331914,-0.11682692,0.04782308,-0.13571422,0.22827948
,-0.23456622,-0.12406075,-0.04466465,0.2311667,-0.12232673,-0.25614032
,-0.36237662,0.11197906,-0.08259534,-0.634934,-0.30822742,-0.93798716
,0.08848668,0.4083417,-0.30910404,0.40950143,0.30815359,0.03266103
,1.03959336,-0.19865537,0.25149713,0.3277561,0.16943092,0.69125975
,0.21721349,-0.30871948,0.88890484,-0.08884043,0.38474549,0.30884107
,-0.40933304,0.30889523,-0.29562966,-0.6271498])
env.unwrapped.set_physics(qpos=goal_qpos, qvel=np.zeros(75))
goal_obs = torch.tensor(env.unwrapped.get_obs()["proprio"].reshape(1,-1), device=model.cfg.device, dtype=torch.float32)
print("goal pose")
media.show_image(env.render())

In [ ]:
z = model.goal_inference(next_obs=goal_obs)


observation, _ = env.reset()
frames = [env.render()]
for i in range(10):
    action = model.act(observation, z, mean=True)
    observation, reward, terminated, truncated, info = env.step(action.cpu().numpy().ravel())
    frames.append(env.render())

media.show_video(frames, fps=30)